In [124]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [125]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [126]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [127]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [128]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [202]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [203]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.379s

OK


In [204]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
transaction = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(transaction)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [205]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

010000000199a24308080ab26e6fb65c4eccfadf76749bb5bfa8cb08f291320b3c21e56f0d0d0000006b4830450221008ed46aa2cf12d6d81065bfabe903670165b538f65ee9a3385e6327d80c66d3b502203124f804410527497329ec4715e18558082d489b218677bd029e7fa306a72236012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff02408af701000000001976a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f88ac80969800000000001976a914507b27411ccf7f16f10297de6cef3f291623eddf88ac00000000


In [208]:
transaction.verify()

True

In [207]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [226]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.287s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [232]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL, little_endian_to_int, hash160, encode_base58_checksum
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

SATOSHIS_PER_BTC = 100000000


# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

passphrase = b'csuh.web@gmail.com I know what you did last summer'
secret = little_endian_to_int(hash256(passphrase))

compressed = True
testnet = True

# create a private key using your secret
private_key = PrivateKey(secret)
public_key = private_key.point

my_address = public_key.address(compressed=compressed, testnet=testnet)
print('my address:', my_address, '\n')

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence

# https://testnet-faucet.mempool.co
# Transaction sent
# TxID: 2dfed6e011e411d1d879b55a68abb1c006d4999c939038a22131c1015917e042
# Address: n1jDwX925pqbwxA3PjLCZaSppawFeh6u2q (uncompressed)
# Amount: 0.01

# https://tbtc.bitaps.com
# TxID: eb59e7694c3b28c3b0c79ef7760f06e6306f5ae2aed5c545da0a9d747b31a2e6
# Address: mfvaMULiWEKRBggtFw1Bo1hy5azxMTZDey (compressed)
# Amount: 0.01

original_amount = int(0.01 * SATOSHIS_PER_BTC)

prev_tx = bytes.fromhex('eb59e7694c3b28c3b0c79ef7760f06e6306f5ae2aed5c545da0a9d747b31a2e6')
prev_index = 0

tx_in = TxIn(prev_tx, prev_index)

# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee
# create a transaction output for the target amount and address
# create a transaction output for the change amount and address
# create the transaction object

target_amount = int(0.6 * original_amount)
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)

fee = int(0.001 * SATOSHIS_PER_BTC)

change_amount = original_amount - target_amount - fee
change_h160 = decode_base58(my_address)
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)

transaction = Tx(
    version=1,
    tx_ins=[tx_in],
    tx_outs=[target_output, change_output], 
    locktime=0, 
    testnet=testnet,
)

print(transaction, '\n')

# sign the one input in the transaction object using the private key
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx

is_success = transaction.sign_input(0, private_key, compressed=compressed)
print('transaction signed successfully:', is_success, '\n')
print('transaction verified:', transaction.verify(), '\n')
print('signed transaction:\n', transaction, '\n')

tx_hex = transaction.serialize().hex()
print('transaction hex:\n', tx_hex, '\n')

my address: mfvaMULiWEKRBggtFw1Bo1hy5azxMTZDey 

tx: 1ea9f8ca1791a053fdc004f84a6e819b5381d516814305a18352847bbdc0b0db
version: 1
tx_ins:
eb59e7694c3b28c3b0c79ef7760f06e6306f5ae2aed5c545da0a9d747b31a2e6:0
tx_outs:
600000:OP_DUP OP_HASH160 ad346f8eb57dee9a37981716e498120ae80e44f7 OP_EQUALVERIFY OP_CHECKSIG
300000:OP_DUP OP_HASH160 0477aae80ef3609795192e05e78215117ce8a179 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0 

transaction signed successfully: True 

transaction verified: True 

signed transaction:
 tx: dc081b66f6e50978b97dd17403757ae9b423eaac413db95694059f4ea680bd77
version: 1
tx_ins:
eb59e7694c3b28c3b0c79ef7760f06e6306f5ae2aed5c545da0a9d747b31a2e6:0
tx_outs:
600000:OP_DUP OP_HASH160 ad346f8eb57dee9a37981716e498120ae80e44f7 OP_EQUALVERIFY OP_CHECKSIG
300000:OP_DUP OP_HASH160 0477aae80ef3609795192e05e78215117ce8a179 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0 

transaction hex:
 0100000001e6a2317b749d0ada45c5d5aee25a6f30e6060f76f79ec7b0c3283b4c69e759eb000000006b483045022100a5c01ac5f483600c3ac

In [230]:
# successfully broadcast!
# TxID: dc081b66f6e50978b97dd17403757ae9b423eaac413db95694059f4ea680bd77

### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [243]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create the first transaction input with the default ScriptSig and
# sequence
prev_tx_1 = bytes.fromhex('a89ca0cad6d2e0a7887cd0abc1b0251af0547e9a4b6e050768f15f7d52be7a42')
prev_index_1 = 0

tx_in_1 = TxIn(prev_tx_1, prev_index_1)

# get the prev_tx and prev_index from the transaction in Exercise 4
# create the second transaction input with the default ScriptSig and
# sequence
prev_tx_2 = bytes.fromhex('dc081b66f6e50978b97dd17403757ae9b423eaac413db95694059f4ea680bd77')
prev_index_2 = 1

tx_in_2 = TxIn(prev_tx_2, prev_index_2)

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee
fee = int(0.001 * SATOSHIS_PER_BTC)
original_amount = tx_in_1.value(testnet) + tx_in_2.value(testnet)
target_amount = original_amount - fee

# create a transaction output for the amount and address
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)

transaction = Tx(
    version=1,
    tx_ins=[tx_in_1, tx_in_2],
    tx_outs=[target_output], 
    locktime=0, 
    testnet=testnet,
)

# sign the first input using the private key
# sign the second input using the private key 
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx

for i, _ in enumerate(transaction.tx_ins):
    print(f'Input {i} signed:', transaction.sign_input(i, private_key))
    
print('Transaction verified:', transaction.verify())
print('Transaction hex:', transaction.serialize().hex())

Input 0 signed: True
Input 1 signed: True
Transaction verified: True
Transaction hex: 0100000002427abe527d5ff16807056e4b9a7e54f01a25b0c1abd07c88a7e0d2d6caa09ca8000000006b483045022100d47cffe314b490c631beae0f694c11e4778d43b47daed06301ae281d0bdc41ae022032381288e68fdd0f9e3a8b2a706d5ed6b0461c78f3fbd0e0fcfb30390238458801210220bc554308700a6882193f7d695f0ce9a8484b906e98c9b2b173958cacc63c87ffffffff77bd80a64e9f059456b93d41acea23b4e97a750374d17db97809e5f6661b08dc010000006a473044022018483ce23b9fa69c7502d5e15f7fed2ccea7c4bd556565c122ffa5e6f4bba8440220446fa83fc640510314a3207c47739b64cc9437ab5ad1fae420df5fd525ff4c5f01210220bc554308700a6882193f7d695f0ce9a8484b906e98c9b2b173958cacc63c87ffffffff01804f1200000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000


In [244]:
# Transaction successfully broadcast!
# TxID: 2db513b1607e8e82dd4d4fb45de4243e0c02780e2ff5423a68dcb6358daab74e